## excel-to-python-course-ch8-case-study-ym

ym's rendition of TalkPython's "Excel to Python Course: ch8-case-study"


### Objectives
- Given last year's sales data, and a commission budget of $1mil, we want to look at how we should distribute sales commission to the sales agents.
- Propose ways to achieve a more balanced commission earnings between sales agent, by adjusting commission rates or region of coverage.


### Data Sources
- 'customer_master.xlsx' contains 3 sets of data, namely customers, transactions, and sales
- https://github.com/talkpython/excel-to-python-course/tree/master/code/ch8-case-study

### Changes
- 16-12-2021 : Started project

In [1]:
import pandas as pd
import xlsxwriter
from pathlib import Path
from datetime import datetime

### File Locations

In [2]:
today = datetime.today()
src_file = Path.cwd() / "data" / "raw" / "customer_master.xlsx"
output_file = Path.cwd() / "data" / "processed" / "customer_processed.xlsx"
output_file2 = Path.cwd() / "data" / "processed" / "customer_processed2.xlsx"

In [3]:
df_trx = pd.read_excel(src_file, sheet_name = "transactions")
df_cust = pd.read_excel(src_file, sheet_name = "customers", dtype={'zip_code':str})
df_salesagt = pd.read_excel(src_file, sheet_name = "sales")

url = "https://github.com/cphalpert/census-regions/blob/master/us%20census%20bureau%20regions%20and%20divisions.csv?raw=True"
regions = pd.read_csv(url, usecols=[1,2])

##### We shall go through each of the 3 datasets one by one.

### (1) Transactions

In [4]:
df_trx

,cust_num,sku,qty,list_price,discount_rate,invoice_price,invoice_num,invoice_date_time,invoice_total
0,LA6029,SW200,4,20000,0.24,15200.0,98105,2019-12-13 14:11:43.828,60800.0
1,EB0265,PS501,4,30000,0.10,27000.0,58436,2019-06-05 23:12:47.344,108000.0
2,EE4079,SW500,1,20000,0.36,12800.0,85825,2019-09-12 03:23:24.309,12800.0
3,YR6861,ACC5144,4,400,0.12,352.0,46422,2019-10-10 15:02:54.590,1408.0
4,WL5283,SW200,1,20000,0.17,16600.0,34838,2019-08-03 11:32:29.245,16600.0
...,...,...,...,...,...,...,...,...,...
1995,XJ1430,SPB1,1,5000,0.19,4050.0,11706,2019-05-09 15:09:09.614,4050.0
1996,AI9833,SW500,3,20000,0.24,15200.0,38703,2019-11-10 03:55:57.038,45600.0
1997,WL5283,SW200,2,20000,0.40,12000.0,48217,2019-10-18 06:00:39.492,24000.0
1998,SM6748,ACC9011,18,400,0.38,248.0,66811,2019-07-24 05:07:14.352,4464.0


In [5]:
df_trx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   cust_num           2000 non-null   object        
 1   sku                2000 non-null   object        
 2   qty                2000 non-null   int64         
 3   list_price         2000 non-null   int64         
 4   discount_rate      2000 non-null   float64       
 5   invoice_price      2000 non-null   float64       
 6   invoice_num        2000 non-null   int64         
 7   invoice_date_time  2000 non-null   datetime64[ns]
 8   invoice_total      2000 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(2)
memory usage: 140.8+ KB


In [6]:
prev_yr_total_sales = df_trx['invoice_total'].sum()
print(f'Total sales for previous year: ${prev_yr_total_sales:,.0f}')

Total sales for previous year: $126,493,662


In [7]:
total_com = 1_000_000
avg_com = total_com / prev_yr_total_sales
print(f'Average commission rate based on previous year\'s sale: {avg_com:.2%}')

Average commission rate based on previous year's sale: 0.79%


In [8]:
df_trx.describe()

,qty,list_price,discount_rate,invoice_price,invoice_num,invoice_total
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,5.336000,15407.700000,0.202315,12279.542000,50350.670500,63246.831000
std,6.072524,9907.746587,0.098452,8098.502539,28755.571742,94703.387591
min,1.000000,400.000000,0.000000,200.000000,43.000000,240.000000
25%,2.000000,5000.000000,0.130000,3950.000000,25356.250000,12400.000000
50%,3.000000,20000.000000,0.200000,14800.000000,50824.000000,32000.000000
75%,4.000000,20000.000000,0.270000,17600.000000,75442.000000,66450.000000
max,24.000000,30000.000000,0.560000,30000.000000,99990.000000,705600.000000


In [9]:
df_trx.describe(include=object)

,cust_num,sku
count,2000,2000
unique,50,12
top,VK4512,SW200
freq,53,338


In [10]:
agg_cols = {'invoice_num' : 'count', 'discount_rate':'mean','qty':'sum', 'invoice_total':'sum'}

In [11]:
df_trx.groupby('sku').agg(agg_cols).reset_index()

,sku,invoice_num,discount_rate,qty,invoice_total
0,ACC0001,73,0.207671,377,118828.0
1,ACC5144,71,0.185493,469,151912.0
2,ACC8222,73,0.197123,443,139876.0
3,ACC9011,84,0.209167,427,133996.0
4,PS403,145,0.199172,662,16130700.0
5,PS501,149,0.201812,784,18609900.0
6,SPA1,137,0.195693,809,3274350.0
7,SPB1,137,0.193796,802,3238100.0
8,SPBC2,141,0.209787,708,2793600.0
9,SW121,320,0.203000,1631,25856400.0


##### Based on the 'transactions' data, we made the below observations:
- Total sales for the previous year was \$126,493,662.
- Average commission rate based on the commission budget of $1mil is 0.79%.
- There were a total of 2000 transactions, between 50 unique customers, and across 12 different products. 
- Some products were more popular than the rest.

### (2) Customers

In [12]:
df_cust

,company_name,channel,zip_code,city,state,account_num,total_sales
0,Universal Technology Vision,retail,22910,Charlottesville,VA,AH5590,1257912
1,East Design Hill,retail,66546,Wakarusa,KS,OL0453,1158564
2,Studio Pacific Galaxy,retail,79698,Abilene,TX,YR6861,1663488
3,Galaxy Building,retail,85275,Mesa,AZ,AS3124,1193560
4,Resource Innovation Future,retail,97013,Canby,OR,DK1362,958040
5,Internet Hill Systems,retail,74360,Picher,OK,KK6153,970886
6,Pacific Hill Application,retail,49862,Munising,MI,MS1866,1271136
7,Net Electronic,retail,42631,Marshes Siding,KY,WA1826,1101414
8,Software Bell Technology,retail,45342,Miamisburg,OH,XJ1430,942044
9,Innovation Net,retail,20390,Washington,DC,NS1312,1010872


In [13]:
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  50 non-null     object
 1   channel       50 non-null     object
 2   zip_code      50 non-null     object
 3   city          50 non-null     object
 4   state         50 non-null     object
 5   account_num   50 non-null     object
 6   total_sales   50 non-null     int64 
dtypes: int64(1), object(6)
memory usage: 2.9+ KB


In [14]:
df_cust.describe().round()

,total_sales
count,50.0
mean,2529873.0
std,2482702.0
min,746216.0
25%,1115702.0
50%,1328859.0
75%,1705738.0
max,9121596.0


In [15]:
df_cust.describe(include=object)

,company_name,channel,zip_code,city,state,account_num
count,50,50,50,50,50,50
unique,50,3,50,48,31,50
top,Universal Technology Vision,retail,22910,Dawson,VA,AH5590
freq,1,38,1,2,4,1


##### Here we see that there are a total of 50 companies in the dataset, all bearing different zipcodes and located across 48 cites/31 states. There are 3 sales channels, with the most common being retail.

In [16]:
pd.pivot_table(df_cust, 
               index = 'company_name', 
               columns = 'channel', 
               values = 'total_sales', 
               aggfunc = 'count', 
               margins = True, 
               fill_value = 0)

channel,partner,reseller,retail,All
company_name,,,,
Advanced Alpha Federated,0,1,0,1
Alpha Electronics,0,0,1,1
Analysis Future Provider,0,0,1,1
Analysis Universal,0,1,0,1
Architecture Application,0,0,1,1
Atlantic Application Star,0,0,1,1
Bell Frontier Resource,0,0,1,1
Contract Electronics Industries,0,0,1,1
Contract Solutions Resource,0,0,1,1


##### This tells us that each company only uses 1 sales channel.

In [17]:
df_cust.groupby('state').agg({'company_name' : 'count', 'total_sales':'sum'}).reset_index().sort_values(by = ['total_sales'],ascending = False)

,state,company_name,total_sales
18,MO,3,10716216
21,NE,2,10360328
10,KS,3,9118012
20,NC,2,8177040
8,ID,1,7853376
9,IL,1,6958500
17,MN,1,6833484
12,LA,1,6557928
7,IA,1,6548220
26,PA,1,6222564


##### The states MO and NE brings in significantly more sales than the rest.

In [18]:
df_cust.sort_values(by='total_sales', ascending=False).head()

,company_name,channel,zip_code,city,state,account_num,total_sales
34,Signal Hill Bell,reseller,68631,Creston,NE,HC3828,9121596
31,West Max Hardware,reseller,64738,Collins,MO,MH1146,8807964
28,South Speed East,reseller,83856,Priest River,ID,KI8637,7853376
32,Telecom North Resource,reseller,28170,Wadesboro,NC,RJ3363,7008576
21,Solutions North,reseller,62520,Dawson,IL,QZ1799,6958500


##### This is probably due to the fact that the top 2 customers are located in these 2 states. Notice also that the top 5 customers are all using the reseller channel.

In [19]:
df_cust.groupby('channel').agg({'company_name' : 'count', 'total_sales':'sum'}).reset_index().sort_values(by = ['total_sales'],ascending = False)

,channel,company_name,total_sales
1,reseller,10,72708276
2,retail,38,46782774
0,partner,2,7002612


##### Even though most companies uses the retail channel, the reseller channel brings in more sales than both retail and partner combined. Let's see if the sales channel has any correlation with the states the companies are in.

In [20]:
pd.pivot_table(df_cust, 
               index = "state", 
               columns = "channel", 
               values = 'total_sales', 
               aggfunc = 'count', 
               margins = True, 
               fill_value =0)

channel,partner,reseller,retail,All
state,,,,
AL,0,0,1,1
AZ,0,0,1,1
CA,0,0,2,2
CT,0,0,2,2
DC,1,0,1,2
FL,1,0,1,2
GA,0,0,1,1
IA,0,1,0,1
ID,0,1,0,1


In [21]:
pd.crosstab(df_cust['state'],
            df_cust['channel'],
            values = df_cust['total_sales'], 
            aggfunc = 'sum',
            normalize = 'index')

channel,partner,reseller,retail
state,,,
AL,0.000000,0.000000,1.000000
AZ,0.000000,0.000000,1.000000
CA,0.000000,0.000000,1.000000
CT,0.000000,0.000000,1.000000
DC,0.790636,0.000000,0.209364
FL,0.753398,0.000000,0.246602
GA,0.000000,0.000000,1.000000
IA,0.000000,1.000000,0.000000
ID,0.000000,1.000000,0.000000


##### The 10 companies that uses the reseller channel are spread out across 10 different states. There does not seem to be any conclusive correlation between the location and the choice of sales channel.

##### The observations made from the 'customers' data can be summarised as below:
- There are 50 companies, located across 41 cities or 31 states.
- There are 3 sales channels, namely retail, reseller, and partner. Each company only uses one sales channel.
- While most companies go through retail, the reseller channel brings in more sales volume than retail and partner combined.
- All top 5 companies (in terms of total sales) uses the reseller channel.
- The top 2 companies are located in MO and NE respectively, which are coincidentially also the top 2 states in terms of total sales volume.


### (3) Sales persons

In [22]:
df_salesagt

,first_name,last_name,region,tenure
0,Shannon,Muniz,NorthEast,5.6
1,Leonard,Malcolm,West,3.8
2,Mona,Sutton,Midwest,5.4
3,Mickey,Tyner,South,0.7


In [23]:
df_salesagt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   first_name  4 non-null      object 
 1   last_name   4 non-null      object 
 2   region      4 non-null      object 
 3   tenure      4 non-null      float64
dtypes: float64(1), object(3)
memory usage: 256.0+ bytes


##### There are a total of 4 sales agents, each covering a different region. Let's see how much commission each of them would have made, if we assume an average commission rate of 0.79% which we calculated earlier.

##### To do this, we need the help of an external table which tells us which region each state is located. This data can be found here: https://github.com/cphalpert/census-regions/blob/master/us%20census%20bureau%20regions%20and%20divisions.csv

##### We will add this input to the top of the file, together with the rest of the inputs.

In [24]:
regions

,State Code,Region
0,AK,West
1,AL,South
2,AR,South
3,AZ,West
4,CA,West
5,CO,West
6,CT,Northeast
7,DC,South
8,DE,South
9,FL,South


##### Let's rename the columns in the region table and also change the spelling of Northeast to be consistent with the dataset we are working with.

In [25]:
regions = regions.rename(columns={'State Code':'state', 'Region':'region'})
regions['region'] = regions['region'].str.replace("Northeast","NorthEast")
regions

,state,region
0,AK,West
1,AL,South
2,AR,South
3,AZ,West
4,CA,West
5,CO,West
6,CT,NorthEast
7,DC,South
8,DE,South
9,FL,South


##### Now we can link each customer to their sales agent.

In [26]:
temp = pd.merge(df_cust,regions)
temp

,company_name,channel,zip_code,city,state,account_num,total_sales,region
0,Universal Technology Vision,retail,22910,Charlottesville,VA,AH5590,1257912,South
1,Contract Electronics Industries,retail,24153,Salem,VA,GG0303,1035050,South
2,Star Interactive,retail,22153,Springfield,VA,UM2244,1541486,South
3,Vision People Solutions,retail,24557,Gretna,VA,WL5283,1299450,South
4,East Design Hill,retail,66546,Wakarusa,KS,OL0453,1158564,Midwest
5,Hardware Adventure Universal,retail,67118,Norwich,KS,GA3939,1163380,Midwest
6,Solutions Universal,reseller,66212,Shawnee Mission,KS,SA4443,6796068,Midwest
7,Studio Pacific Galaxy,retail,79698,Abilene,TX,YR6861,1663488,South
8,Virtual Vision Data,retail,77501,Pasadena,TX,YA6348,1440886,South
9,Galaxy Building,retail,85275,Mesa,AZ,AS3124,1193560,West


In [27]:
cust_salesagt = pd.merge(temp, df_salesagt, how='left')

In [28]:
cust_salesagt

,company_name,channel,zip_code,city,state,account_num,total_sales,region,first_name,last_name,tenure
0,Universal Technology Vision,retail,22910,Charlottesville,VA,AH5590,1257912,South,Mickey,Tyner,0.7
1,Contract Electronics Industries,retail,24153,Salem,VA,GG0303,1035050,South,Mickey,Tyner,0.7
2,Star Interactive,retail,22153,Springfield,VA,UM2244,1541486,South,Mickey,Tyner,0.7
3,Vision People Solutions,retail,24557,Gretna,VA,WL5283,1299450,South,Mickey,Tyner,0.7
4,East Design Hill,retail,66546,Wakarusa,KS,OL0453,1158564,Midwest,Mona,Sutton,5.4
5,Hardware Adventure Universal,retail,67118,Norwich,KS,GA3939,1163380,Midwest,Mona,Sutton,5.4
6,Solutions Universal,reseller,66212,Shawnee Mission,KS,SA4443,6796068,Midwest,Mona,Sutton,5.4
7,Studio Pacific Galaxy,retail,79698,Abilene,TX,YR6861,1663488,South,Mickey,Tyner,0.7
8,Virtual Vision Data,retail,77501,Pasadena,TX,YA6348,1440886,South,Mickey,Tyner,0.7
9,Galaxy Building,retail,85275,Mesa,AZ,AS3124,1193560,West,Leonard,Malcolm,3.8


##### Let's bring in the transaction data as well.

In [29]:
detailed_trx = pd.merge(df_trx, cust_salesagt, left_on = "cust_num", right_on = "account_num", how="left")
detailed_trx

,cust_num,sku,qty,list_price,discount_rate,invoice_price,invoice_num,invoice_date_time,invoice_total,company_name,channel,zip_code,city,state,account_num,total_sales,region,first_name,last_name,tenure
0,LA6029,SW200,4,20000,0.24,15200.0,98105,2019-12-13 14:11:43.828,60800.0,Bell Frontier Resource,retail,95172,San Jose,CA,LA6029,1719822,West,Leonard,Malcolm,3.8
1,EB0265,PS501,4,30000,0.10,27000.0,58436,2019-06-05 23:12:47.344,108000.0,Speed Resource Vision,retail,64074,Napoleon,MO,EB0265,746216,Midwest,Mona,Sutton,5.4
2,EE4079,SW500,1,20000,0.36,12800.0,85825,2019-09-12 03:23:24.309,12800.0,Venture Construction,retail,06016,Broad Brook,CT,EE4079,1559544,NorthEast,Shannon,Muniz,5.6
3,YR6861,ACC5144,4,400,0.12,352.0,46422,2019-10-10 15:02:54.590,1408.0,Studio Pacific Galaxy,retail,79698,Abilene,TX,YR6861,1663488,South,Mickey,Tyner,0.7
4,WL5283,SW200,1,20000,0.17,16600.0,34838,2019-08-03 11:32:29.245,16600.0,Vision People Solutions,retail,24557,Gretna,VA,WL5283,1299450,South,Mickey,Tyner,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,XJ1430,SPB1,1,5000,0.19,4050.0,11706,2019-05-09 15:09:09.614,4050.0,Software Bell Technology,retail,45342,Miamisburg,OH,XJ1430,942044,Midwest,Mona,Sutton,5.4
1996,AI9833,SW500,3,20000,0.24,15200.0,38703,2019-11-10 03:55:57.038,45600.0,Resource Adventure Internet,retail,49752,Kinross,MI,AI9833,1580248,Midwest,Mona,Sutton,5.4
1997,WL5283,SW200,2,20000,0.40,12000.0,48217,2019-10-18 06:00:39.492,24000.0,Vision People Solutions,retail,24557,Gretna,VA,WL5283,1299450,South,Mickey,Tyner,0.7
1998,SM6748,ACC9011,18,400,0.38,248.0,66811,2019-07-24 05:07:14.352,4464.0,Advanced Alpha Federated,reseller,56023,Delavan,MN,SM6748,6833484,Midwest,Mona,Sutton,5.4


##### Let's clean up this table by removing the unncessary columns.

In [30]:
final_data = detailed_trx.iloc[:,[1,2,8,9,10,13,16,17,18,19]].copy()
final_data

,sku,qty,invoice_total,company_name,channel,state,region,first_name,last_name,tenure
0,SW200,4,60800.0,Bell Frontier Resource,retail,CA,West,Leonard,Malcolm,3.8
1,PS501,4,108000.0,Speed Resource Vision,retail,MO,Midwest,Mona,Sutton,5.4
2,SW500,1,12800.0,Venture Construction,retail,CT,NorthEast,Shannon,Muniz,5.6
3,ACC5144,4,1408.0,Studio Pacific Galaxy,retail,TX,South,Mickey,Tyner,0.7
4,SW200,1,16600.0,Vision People Solutions,retail,VA,South,Mickey,Tyner,0.7
...,...,...,...,...,...,...,...,...,...,...
1995,SPB1,1,4050.0,Software Bell Technology,retail,OH,Midwest,Mona,Sutton,5.4
1996,SW500,3,45600.0,Resource Adventure Internet,retail,MI,Midwest,Mona,Sutton,5.4
1997,SW200,2,24000.0,Vision People Solutions,retail,VA,South,Mickey,Tyner,0.7
1998,ACC9011,18,4464.0,Advanced Alpha Federated,reseller,MN,Midwest,Mona,Sutton,5.4


In [31]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sku            2000 non-null   object 
 1   qty            2000 non-null   int64  
 2   invoice_total  2000 non-null   float64
 3   company_name   2000 non-null   object 
 4   channel        2000 non-null   object 
 5   state          2000 non-null   object 
 6   region         2000 non-null   object 
 7   first_name     2000 non-null   object 
 8   last_name      2000 non-null   object 
 9   tenure         2000 non-null   float64
dtypes: float64(2), int64(1), object(7)
memory usage: 171.9+ KB


##### Let's add in the commissions.

In [32]:
final_data['comm_rate'] = avg_com
final_data['comm'] = round(final_data['invoice_total'] * avg_com,2)
final_data

,sku,qty,invoice_total,company_name,channel,state,region,first_name,last_name,tenure,comm_rate,comm
0,SW200,4,60800.0,Bell Frontier Resource,retail,CA,West,Leonard,Malcolm,3.8,0.007906,480.66
1,PS501,4,108000.0,Speed Resource Vision,retail,MO,Midwest,Mona,Sutton,5.4,0.007906,853.80
2,SW500,1,12800.0,Venture Construction,retail,CT,NorthEast,Shannon,Muniz,5.6,0.007906,101.19
3,ACC5144,4,1408.0,Studio Pacific Galaxy,retail,TX,South,Mickey,Tyner,0.7,0.007906,11.13
4,SW200,1,16600.0,Vision People Solutions,retail,VA,South,Mickey,Tyner,0.7,0.007906,131.23
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,SPB1,1,4050.0,Software Bell Technology,retail,OH,Midwest,Mona,Sutton,5.4,0.007906,32.02
1996,SW500,3,45600.0,Resource Adventure Internet,retail,MI,Midwest,Mona,Sutton,5.4,0.007906,360.49
1997,SW200,2,24000.0,Vision People Solutions,retail,VA,South,Mickey,Tyner,0.7,0.007906,189.73
1998,ACC9011,18,4464.0,Advanced Alpha Federated,reseller,MN,Midwest,Mona,Sutton,5.4,0.007906,35.29


##### Let's do a brief check whether the datasets were merged correctly, by counter checking that the total comm and the total sales.

In [33]:
final_data['comm'].sum()

1000000.05

In [34]:
final_data['invoice_total'].sum() == prev_yr_total_sales

True

In [35]:
final_data.groupby(['last_name']).agg({'invoice_total':'sum', 'tenure':'last'}).reset_index()

,last_name,invoice_total,tenure
0,Malcolm,12909772.0,3.8
1,Muniz,12850364.0,5.6
2,Sutton,55940194.0,5.4
3,Tyner,44793332.0,0.7


In [36]:
final_data.groupby(['last_name','channel']).agg({'invoice_total':'sum', 'tenure':'last'})

invoice_total  tenure
last_name channel                        
Malcolm   reseller      7853376.0     3.8
          retail        5056396.0     3.8
Muniz     reseller      6222564.0     5.6
          retail        6627800.0     5.6
Sutton    reseller     45065832.0     5.4
          retail       10874362.0     5.4
Tyner     partner       7002612.0     0.7
          reseller     13566504.0     0.7
          retail       24224216.0     0.7

##### Based on the sales agents' data, the following observations can be made:
- There doesn't seem to be any correlation between years of experience and the total sales brought in.
- All 4 agents have a share of sales via reseller and retail.
- The agent who brought in the most sales, Sutton, has a very high sales volume coming in via the reseller channel.
- Only Tyner has sales coming in via the partner channel.

#### Now, we should have all the data we need to analyse the commission rates. Let's save this final data table into a new excel file.

In [37]:
final_data.to_excel(output_file, sheet_name = "final_data", index = False)

In [38]:
# # why doesn't this work???
writer = pd.ExcelWriter(output_file2)
final_data.to_excel(writer, index = False, sheet_name="final_data")
"""
Solution 1 - Enable original code to work:
  1. Need to add `writer.close()`.
  2. Based on the documentation here: https://pandas.pydata.org/docs/reference/api/pandas.ExcelWriter.html
  3. i.e. "... Otherwise, call close() to save and close any opened file handles."
"""
writer.close() # <-- need to close

"""
Solution 2 - Alternate solution using Context Manager.
  1. Before going any further, note that context manager might be a
     new (advance) concept to grasp. Just know how to use it for now.
  2. But basically, it handles the "open()" and "close()" of classes.
  3. Note the solution below doesn't need for you to add `writer.close()`.
  4. As a best practice, and as per documentation, using a Context Manager is
     good: "The writer should be used as a context manager. ..."
"""
with pd.ExcelWriter("data/processed/using_context_manager.xlsx") as writer_cm:
    final_data.to_excel(writer_cm)

"""
Oh, just noticed, yes the solution below uses a context manager :thumbs_up:
"""

In [39]:
# with pd.ExcelWriter(output_file2, engine="xlsxwriter") as writer:
#     final_data.to_excel(writer, index = False, sheet_name="final_data")
#     numformat = xlsxwriter.Workbook(output_file2).add_format({'num_format':',.2f'})
#     writer.sheets['final_data'].set_column('C:C',14, numformat) ###cant give me correct format, only bold
#     writer.sheets['final_data'].set_column('D:D',28)